# Importar librerias

## Instalacion librerias

In [71]:
import pandas as pd
import re #Sirve para string

# Importación dataset

## Instalacion de gdown


In [ ]:
import pandas as pd
df = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
df.head(10)

In [ ]:
print(df.head(10))

In [ ]:
df.iloc[:,:14].head()


In [ ]:
df.iloc[:,14:].head()

In [ ]:
df.iloc[:,14:].tail()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# Exploración de primeros 50 valores únicos solo para variables categóricas
for col in df.select_dtypes(include='object').columns:
    print(f"--- {col} ---")
    print(df[col].dropna().unique()[:50])  # Primeros 50 valores únicos
    print("\n")



In [ ]:
# Vamos a renombrar las columnas para hacerlas más comprensibles, se trata de un cambio en el Diccionario
rename_columns = {
    'DR_NO': 'id_report',
    'Date Rptd': 'date_report',
    'DATE OCC': 'date_ocurrance',
    'TIME OCC': 'time_ocurrance',
    'AREA': 'area_code',
    'AREA NAME': 'area_name',
    'Rpt Dist No': 'district',
    'Part 1-2': 'crime_severity',
    'Crm Cd': 'crime_code',
    'Crm Cd Desc': 'crime_desc',
    'Mocodes': 'crime_mo',
    'Vict Age': 'age_victim',
    'Vict Sex': 'sex_victim',
    'Vict Descent': 'descent_victim',
    'Premis Cd': 'loc_code',
    'Premis Desc': 'loc_desc',
    'Weapon Used Cd': 'weapon_code',
    'Weapon Desc': 'weapon_desc',
    'Status': 'case_status_code',
    'Status Desc': 'case_status_desc',
    'Crm Cd 1': 'crime_code_1',
    'Crm Cd 2': 'crime_code_2',
    'Crm Cd 3': 'crime_code_3',
    'Crm Cd 4': 'crime_code_4',
    'LOCATION': 'location',
    'Cross Street': 'cross_street',
    'LAT': 'latitude',
    'LON': 'longitude'
}

# Renombrar las columnas
df.rename(columns=rename_columns, inplace=True)

# Confirmar el cambio
df.head()


In [ ]:
# Variables a explorar
columns_to_explore = [
    'area_code',
    'area_name',
    'district',
    'crime_severity',
    'crime_code',
    'crime_desc',
    'sex_victim',
    'descent_victim',
    'loc_code',
    'loc_desc',
    'weapon_code',
    'weapon_desc',
    'case_status_code',
    'case_status_desc'
]

# Mostrar TODOS los valores únicos para esas columnas
for col in columns_to_explore:
    if col in df.columns:
        print(f"--- {col} ---")
        print(sorted(df[col].dropna().unique()))
        print("\n")
    else:
        print(f"Column {col} not found in the dataframe.\n")


## **Vamos a trabajar en investigar la relación de códigos varios en el dataset. **

# 📄 Unique Values Summary

## area_code
`1, 2, 3, ..., 21`

## area_name
`'77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck', 'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic', 'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys', 'West LA', 'West Valley', 'Wilshire'`

## district
`101, 105, 109, 111, ..., 2199`  
(Muchos valores únicos de distrito, códigos de zonas específicas)

## crime_severity
`1, 2`

## crime_code
`110, 113, 121, 122, 210, 220, 230, 231, ..., 956`

## crime_desc
`'ARSON', 'ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER', 'BATTERY - SIMPLE ASSAULT', 'BIKE - STOLEN', 'BURGLARY', 'BURGLARY FROM VEHICLE', ..., 'WEAPONS POSSESSION/BOMBING'`
(Más de 100 tipos de crímenes diferentes)

## crime_mo
`Códigos tipo '0344 1822 1402', '0930', '0913 2024' etc.`  
(Modus Operandi combinados por código)

## sex_victim
`'-', 'F', 'H', 'M', 'X'`
(Indica sexo: Femenino, Masculino, Indeterminado, No reportado)

## descent_victim
`'-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z'`
(Origen étnico de la víctima)

## loc_code
`101.0, 102.0, 103.0, ..., 976.0`
(Muchos códigos de localización)

## loc_desc
`'STREET', 'BUS STOP', 'PARKING LOT', 'SINGLE FAMILY DWELLING', 'SCHOOL', 'HOSPITAL', 'HOTEL', 'MARKET', ..., 'YARD (RESIDENTIAL/BUSINESS)'`
(Lugares donde ocurrieron los crímenes)

## weapon_code
`101.0, 102.0, ..., 516.0`

## weapon_desc
`'HAND GUN', 'KNIFE WITH BLADE 6INCHES OR LESS', 'STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)', 'UNKNOWN FIREARM', 'VERBAL THREAT', ...`
(Descripción de armas usadas)

## case_status_code
`'AA', 'AO', 'CC', 'IC', 'JA', 'JO'`
(Códigos de estatus del caso)

## case_status_desc
`'Adult Arrest', 'Adult Other', 'Invest Cont', 'Juv Arrest', 'Juv Other', 'UNK'`
(Descripción del estatus del caso)


##1. Investigación de las áreas.

In [ ]:
# Primero aseguramos que district sea string para extraer area_code
df['district'] = df['district'].astype(str)

# Sacamos el primer dígito como área (puedes usar 1 o 2 dígitos según precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Mapeo de area_code a area_name
area_mapping = {
    1: '77th Street',
    2: 'Central',
    3: 'Devonshire',
    4: 'Foothill',
    5: 'Harbor',
    6: 'Hollenbeck',
    7: 'Hollywood',
    8: 'Mission',
    9: 'N Hollywood',
    10: 'Newton',
    11: 'Northeast',
    12: 'Olympic',
    13: 'Pacific',
    14: 'Rampart',
    15: 'Southeast',
    16: 'Southwest',
    17: 'Topanga',
    18: 'Van Nuys',
    19: 'West LA',
    20: 'West Valley',
    21: 'Wilshire'
}

# Asignamos nombre al área
df['area_name'] = df['area_code'].map(area_mapping)

# Verificamos si el mapeo está correcto
print(df[['district', 'area_code', 'area_name']].head())

# Contar registros por area_name
print(df['area_name'].value_counts())


In [ ]:
import pandas as pd

# Definimos el listado completo de áreas (en el orden correcto)
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# Hacemos el conteo de registros por area_name
area_counts = df['area_name'].value_counts()

# Aseguramos que todos los nombres estén incluidos (incluso los que faltan)
area_counts = area_counts.reindex(area_names, fill_value=0)

# Ahora, buscamos las áreas con 0 crímenes
areas_sin_crimen = area_counts[area_counts == 0]

# Mostrar áreas sin crímenes
print("Áreas donde no hubo crímenes registrados:")
print(areas_sin_crimen)

In [ ]:
# Definimos las 21 áreas oficiales
area_names = [
    '77th Street', 'Central', 'Devonshire', 'Foothill', 'Harbor', 'Hollenbeck',
    'Hollywood', 'Mission', 'N Hollywood', 'Newton', 'Northeast', 'Olympic',
    'Pacific', 'Rampart', 'Southeast', 'Southwest', 'Topanga', 'Van Nuys',
    'West LA', 'West Valley', 'Wilshire'
]

# 1. Conteo de crímenes por área
area_counts = df['area_name'].value_counts()

# 2. Asegurarnos que todas las áreas estén (rellenar con 0 si no existen)
area_counts = area_counts.reindex(area_names, fill_value=0)

# 3. Detectar áreas sin crímenes
areas_sin_crimen = area_counts[area_counts == 0].index.tolist()

print("Áreas donde no hubo crímenes:")
print(areas_sin_crimen)

# 4. Filtrar el df para filas con esas áreas (debería estar vacío, o muy pequeño si algo raro pasó)
df_sin_crimenes = df[df['area_name'].isin(areas_sin_crimen)]

# 5. Mostrar las filas
print(df_sin_crimenes)


Resumen:
9 áreas sí tienen crímenes (las que viste antes: 77th Street, Central, Devonshire, etc.).

12 áreas no tienen crímenes en tus datos actuales.

No hay errores de mapeo en district, area_code ni area_name.

## 2. Investigación Distritos.

In [ ]:
# Mostrar los valores únicos de la columna 'district'
if 'district' in df.columns:
    print("--- district ---")
    print(sorted(df['district'].dropna().unique()))
    print("\n")
else:
    print("Column 'district' not found in the dataframe.\n")

In [ ]:
# Asegurar que 'district' es string para extraer prefijos fácilmente
df['district'] = df['district'].astype(str)

# Extraer primer dígito (o primeros dos dígitos si quieres más precisión)
df['area_code'] = df['district'].str[0].astype(int)

# Opcionalmente, si quieres extraer dos primeros dígitos:
# df['area_code'] = df['district'].str[:2].astype(int)

# Definir los áreas válidas (1 al 21)
valid_areas = list(range(1, 22))

# Marcar si es un área válida
df['area_valid'] = df['area_code'].isin(valid_areas)

# Verificar resultados
print(df['area_valid'].value_counts())

# Ver ejemplos
print(df[['district', 'area_code', 'area_valid']].head())


In [ ]:
# Contar cuántos registros NO son válidos
invalid_count = (~df['area_valid']).sum()

print(f"Cantidad de registros con area_valid == False: {invalid_count}")


##3. Relación entre crime_code y crime_desc

Los crime_code y crime_desc que tienes provienen directamente de los reportes oficiales del LAPD (Los Angeles Police Department).

Cada "crime_code" es un tipo de crimen específico, basado en su severidad y en las leyes de California.

No es una clasificación inventada, es estándar dentro del departamento, usado para categorizar las denuncias.

In [ ]:
# Comprobar si un mismo crime_code tiene varias descripciones
crime_mapping = df[['crime_code', 'crime_desc']].drop_duplicates()

# Verificar duplicados de crime_code
duplicates = crime_mapping.duplicated(subset=['crime_code'], keep=False)
crime_mapping[duplicates]

Si no salen filas, la relación es uno a uno ✅.

In [ ]:
# Crear tabla resumen de códigos de crimen y su descripción
crime_table = df[['crime_code', 'crime_desc']].drop_duplicates().sort_values('crime_code')

# Mostrar la tabla
crime_table.reset_index(drop=True, inplace=True)
crime_table


In [33]:
crime_table.to_csv('crime_code_description.csv', index=False)

Vamos a generar una agrupación de todos estos delitos en base al standard UCR (Uniform Crime Reporting)

In [ ]:
import pandas as pd

# Suponemos que tienes un DataFrame llamado 'df' con las columnas 'crime_code' y 'crime_desc'

# Mapeo de crime_codes a categorías UCR
ucr_mapping = {
    # Part I - Violent Crimes
    110: 'HOMICIDE',
    113: 'MANSLAUGHTER',
    121: 'RAPE',
    122: 'ATTEMPTED RAPE',
    815: 'SEXUAL PENETRATION',
    820: 'ORAL COPULATION',
    821: 'SODOMY',
    210: 'ROBBERY',
    220: 'ROBBERY ATTEMPTED',
    230: 'AGGRAVATED ASSAULT',
    231: 'AGGRAVATED ASSAULT',
    235: 'AGGRAVATED ASSAULT',
    236: 'AGGRAVATED ASSAULT',
    250: 'AGGRAVATED ASSAULT',
    251: 'AGGRAVATED ASSAULT',
    761: 'AGGRAVATED ASSAULT',
    926: 'AGGRAVATED ASSAULT',
    435: 'SIMPLE ASSAULT',
    436: 'SIMPLE ASSAULT',
    437: 'SIMPLE ASSAULT',
    622: 'SIMPLE ASSAULT',
    623: 'SIMPLE ASSAULT',
    624: 'SIMPLE ASSAULT',
    625: 'SIMPLE ASSAULT',
    626: 'SIMPLE ASSAULT',
    627: 'SIMPLE ASSAULT',
    647: 'SIMPLE ASSAULT',
    763: 'SIMPLE ASSAULT',
    928: 'SIMPLE ASSAULT',
    930: 'SIMPLE ASSAULT',

    # Part I - Property Crimes
    310: 'BURGLARY',
    320: 'BURGLARY',
    510: 'MOTOR VEHICLE THEFT',
    520: 'MOTOR VEHICLE THEFT',
    433: 'MOTOR VEHICLE THEFT',
    330: 'THEFT FROM VEHICLE',
    331: 'THEFT FROM VEHICLE',
    410: 'THEFT FROM VEHICLE',
    420: 'THEFT FROM VEHICLE',
    421: 'THEFT FROM VEHICLE',
    350: 'PERSONAL THEFT',
    351: 'PERSONAL THEFT',
    352: 'PERSONAL THEFT',
    353: 'PERSONAL THEFT',
    450: 'PERSONAL THEFT',
    451: 'PERSONAL THEFT',
    452: 'PERSONAL THEFT',
    453: 'PERSONAL THEFT',
    341: 'OTHER THEFT',
    343: 'OTHER THEFT',
    345: 'OTHER THEFT',
    440: 'OTHER THEFT',
    441: 'OTHER THEFT',
    442: 'OTHER THEFT',
    443: 'OTHER THEFT',
    444: 'OTHER THEFT',
    445: 'OTHER THEFT',
    470: 'OTHER THEFT',
    471: 'OTHER THEFT',
    472: 'OTHER THEFT',
    473: 'OTHER THEFT',
    474: 'OTHER THEFT',
    475: 'OTHER THEFT',
    480: 'OTHER THEFT',
    485: 'OTHER THEFT',
    487: 'OTHER THEFT',
    491: 'OTHER THEFT',

    # Part II - Other Crimes
    500: 'DRUG OFFENSES',
    501: 'DRUG OFFENSES',
    502: 'DRUG OFFENSES',
    503: 'DRUG OFFENSES',
    504: 'DRUG OFFENSES',
    505: 'DRUG OFFENSES',
    506: 'DRUG OFFENSES',
    600: 'SIMPLE ASSAULT',
    601: 'SIMPLE ASSAULT',
    602: 'SIMPLE ASSAULT',
    603: 'SIMPLE ASSAULT',
    700: 'VANDALISM',
    701: 'VANDALISM',
    702: 'WEAPONS OFFENSE',
    703: 'WEAPONS OFFENSE',
    704: 'WEAPONS OFFENSE',
    705: 'WEAPONS OFFENSE',
    800: 'SEX OFFENSES (NON-RAPE)',
    801: 'SEX OFFENSES (NON-RAPE)',
    802: 'SEX OFFENSES (NON-RAPE)',
    803: 'SEX OFFENSES (NON-RAPE)',
    900: 'OTHER OFFENSES',
    901: 'OTHER OFFENSES',
    902: 'OTHER OFFENSES',
    903: 'OTHER OFFENSES',
    904: 'OTHER OFFENSES',
    905: 'OTHER OFFENSES',
    906: 'OTHER OFFENSES',
    907: 'OTHER OFFENSES'
}

# Crear nueva columna 'crime_ucr' basada en el mapping
df['crime_ucr'] = df['crime_code'].map(ucr_mapping).fillna('UNKNOWN')

# Si quieres ver los resultados
print(df[['crime_code', 'crime_desc', 'crime_ucr']].head())


In [ ]:
df.columns

## 4. Investigación de Sexo.

In [ ]:
# 1. Primero, corregimos H -> M
df['sex_victim'] = df['sex_victim'].replace('H', 'M')

# 2. Definimos la lista válida final
valores_validos_sexo = ['-', 'F', 'M', 'X']

# 3. Comprobamos de nuevo si queda todo limpio
print("Valores únicos en sex_victim después de la corrección:")
print(df['sex_victim'].unique())

# 4. Opcional: identificar si queda algo inválido
valores_invalidos = df[~df['sex_victim'].isin(valores_validos_sexo)]

print(f"\nCantidad de registros con sex_victim inválido después de corregir: {len(valores_invalidos)}")


In [ ]:
# Reemplazar nulos por '-'
df['sex_victim'] = df['sex_victim'].fillna('-')

# Volver a verificar
print("Valores únicos en sex_victim después de rellenar NaN:")
print(df['sex_victim'].unique())

In [ ]:
# 1. Conteo de sex_victim
conteo_sexo = df['sex_victim'].value_counts()

# 2. Porcentaje de sex_victim
porcentaje_sexo = df['sex_victim'].value_counts(normalize=True) * 100

# 3. Mostrar resultados
resultado = pd.DataFrame({
    'Cantidad': conteo_sexo,
    'Porcentaje (%)': porcentaje_sexo.round(2)
})

print(resultado)

## 5. Origen

In [ ]:
# 1. Definimos los valores válidos
valores_validos_descent = ['-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 
                           'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z']

# 2. Verificamos los valores únicos en el dataset
print("Valores únicos en descent_victim:")
print(df['descent_victim'].unique())

# 3. Detectamos valores inválidos
valores_invalidos = df[~df['descent_victim'].isin(valores_validos_descent)]

# 4. Mostramos las filas con descent_victim inválido
print("\nFilas con valores inválidos en descent_victim:")
print(valores_invalidos[['id_report', 'descent_victim']])

# 5. Cantidad de registros inválidos
print(f"\nCantidad de registros con descent_victim inválido: {len(valores_invalidos)}")

In [66]:
# Reemplazar NaN por '-'
df['descent_victim'] = df['descent_victim'].fillna('-')

In [ ]:
# 1. Definir los valores válidos de razas
valores_validos_descent = ['-', 'A', 'B', 'C', 'D', 'F', 'G', 'H', 'I', 'J', 
                           'K', 'L', 'O', 'P', 'S', 'U', 'V', 'W', 'X', 'Z']

# 2. Rellenar nulos si es necesario
df['descent_victim'] = df['descent_victim'].fillna('-')

# 3. Conteo de víctimas por raza
conteo_raza = df['descent_victim'].value_counts()

# 4. Porcentaje por raza
porcentaje_raza = df['descent_victim'].value_counts(normalize=True) * 100

# 5. Crear el DataFrame resultado
resultado_raza = pd.DataFrame({
    'Cantidad': conteo_raza,
    'Porcentaje (%)': porcentaje_raza.round(2)
})

# 6. Mostrar resultados
print(resultado_raza)

### id_report

In [ ]:
df['id_report'].info()

In [ ]:
df.iloc[:, :5].head(10)

In [ ]:
df.info()

In [1]:
df.loc[df['id_report']<9999999]

NameError: name 'df' is not defined

### date_report

In [58]:
df['date_report']=pd.to_datetime(df['date_report'])

In [ ]:
df['date_report'].head()

In [ ]:
df['date_report'].info()

### date_ocurrance

In [ ]:
df['date_ocurrance']=pd.to_datetime(df['date_ocurrance'])

In [ ]:
df['date_ocurrance'].head()

In [ ]:
df['date_report'].info()

### time_ocurrance

In [60]:
df['time_ocurrance'] = df['time_ocurrance'].astype(str).str.zfill(4) #Asi pongo 4 cifras en la columna

In [61]:
df['time_ocurrance'] = df['time_ocurrance'].str[:-2] + ":" + df['time_ocurrance'].str[-2:] # Aqui converti a string y luego lo separe lo 2 primero numeros, luego : y luego los 2 ultimos

In [62]:
df['time_ocurrance'] = pd.to_datetime(df['time_ocurrance'], format ='%H:%M').dt.time

In [ ]:
df.head(10)

In [ ]:
df.info()

### area_code

### area_name

### age_victim

### location

In [ ]:
df['location'].unique()

In [72]:
df['location'] = df['location'].apply(lambda x: re.sub(r'\s+', ' ', str(x)))    #la funcion re.sub es para reemplazar (lo que quieres reemplazar, por lo que lo vas a reemplazar)

In [80]:
df['location'] = df['location'].apply(str.strip).apply(str.lower)    # Para quitar espacios y ponerlo en minuscula

In [ ]:
df.head()

In [ ]:
df.info()

### cross_street

In [ ]:
df['cross_street'].unique()

In [90]:
df['cross_street'] = df['cross_street'].astype(str)

In [92]:
df['cross_street'] = df['cross_street'].apply(str.strip).apply(str.lower)    # Para quitar espacios y ponerlo en minuscula

In [ ]:
df.tail()

### latitude

### longitude

### Todo

In [58]:
features_explore_JA = ['id_report','date_report','date_ocurrance','time_ocurrance','area_code','area_name','age_victim','location','cross_street','latitude','longitude']

In [ ]:
df[features_explore_JA].info()

In [ ]:
df[features_explore_JA].describe()

In [ ]:
df_categorical = df[features_explore_JA].select_dtypes(include='object')
df_categorical.describe()

In [ ]:
df.loc[df['id_report']<9999999]

In [ ]:
df['age_victim'].loc[df['age_victim']<0]

In [ ]:
df[features_explore_JA].